In [1]:
!python.exe -m pip install --upgrade pip
%pip install -q langchain-elasticsearch
%pip install -q sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from uuid import uuid4
from langchain_core.documents import Document
from elasticsearch import Elasticsearch
from langchain_elasticsearch import ElasticsearchStore
from langchain_community.retrievers import ElasticSearchBM25Retriever
from langchain.embeddings import HuggingFaceEmbeddings

c:\Program Files\Python311\Lib\importlib\__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


In [2]:
with open(
    file="ТИУ_Графовая_База_знаний.txt",
    mode='r',
    encoding='utf-8'
) as file:
    text = file.read()
    
len(text)

271386

In [3]:
import re


entity_pattern = re.compile(r'\(entity\|([^|]+)\|([^|]+)\|([^|]+)\)')

relationship_pattern = re.compile(r'\(relationship\|([^|]+)\|([^|]+)\|([^|]+)\|([^|]+)\)')

In [4]:
entities = []
for match in entity_pattern.finditer(text):
    entity_name, entity_type, entity_description = match.groups()
    entities.append({
        "name": entity_name.strip(),
        "type": entity_type.strip(),
        "description": entity_description.strip()
    })
    
len(entities)    

1592

In [5]:
entities[101]

{'name': 'РАДИАЦИОННАЯ_КАТАСТРОФА',
 'type': 'СОБЫТИЕ',
 'description': 'происшествие, связанное с радиационным воздействием'}

In [8]:
%pip install -q tf-keras

Note: you may need to restart the kernel to use updated packages.


In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="ai-forever/sbert_large_nlu_ru",
    model_kwargs={"device": "cpu"},
    encode_kwargs={'normalize_embeddings': False},
)

C:\Users\andre\AppData\Local\Temp\ipykernel_23944\527724830.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [7]:
documents = []
for entity in entities:
    document = Document(
        page_content=entity["name"],
        metadata={
            "type": entity["type"], 
            "description": entity["description"]
        }
    )
    documents.append(document)

In [8]:
batch_size = 100
batches = []

for i in range(0, len(documents), batch_size):
    batches.append(documents[i:i+batch_size])

print(f"Всего создано {len(batches)} батчей.")

Всего создано 16 батчей.


In [16]:
es = Elasticsearch(
    hosts="https://elasticsearch-pv2s-production.up.railway.app/",
    basic_auth=("elastic", "2qq0lvgc89lwh6z5jp2q9280dcneaf95")
)

In [ ]:
# es.indices.delete(index="nodes_embeddings")

ObjectApiResponse({'acknowledged': True})

In [17]:
elastic_store = ElasticsearchStore(
    es_url="https://elasticsearch-pv2s-production.up.railway.app/",
    index_name="nodes_embeddings",
    embedding=embeddings,
    es_user="elastic",
    es_password="2qq0lvgc89lwh6z5jp2q9280dcneaf95",
)

In [20]:
for batch in batches:
    elastic_store.add_documents(batch)

ConnectionTimeout: Connection timed out

In [16]:
docs = elastic_store.similarity_search("Раскажи о всех филиалах в ТИУ")
docs

[Document(metadata={'type': 'ВУЗ', 'description': 'Отделение основного вуза'}, page_content='ФИЛИАЛ'),
 Document(metadata={'type': 'ВУЗ', 'description': 'Отделение основного вуза'}, page_content='ФИЛИАЛ'),
 Document(metadata={'type': 'ИНСТИТУТ', 'description': 'Подразделение института'}, page_content='Филиал'),
 Document(metadata={'type': 'ОБРАЗОВАТЕЛЬНОЕ_УЧРЕЖДЕНИЕ', 'description': 'Филиал образовательного учреждения'}, page_content='ФИЛИАЛ')]

In [17]:
es.indices.delete(index="nodes_documents")

ObjectApiResponse({'acknowledged': True})

In [18]:
bm25_retriever = ElasticSearchBM25Retriever(
    client=es, 
    index_name="nodes_documents"
)

In [19]:
for batch in batches:
    bm25_retriever.add_texts([document.page_content for document in batch])

In [20]:
docs = bm25_retriever.invoke("Проходной балл на нефтегазовое дело")
docs

[Document(metadata={}, page_content='Нефтегазовое дело'),
 Document(metadata={}, page_content='НЕФТЕГАЗОВОЕ ДЕЛО'),
 Document(metadata={}, page_content='Проходной балл'),
 Document(metadata={}, page_content='Проходной балл'),
 Document(metadata={}, page_content='ПРОХОДНОЙ БАЛЛ'),
 Document(metadata={}, page_content='Проходной балл'),
 Document(metadata={}, page_content='Проходной балл'),
 Document(metadata={}, page_content='Проходной балл'),
 Document(metadata={}, page_content='ПРОХОДНОЙ БАЛЛ'),
 Document(metadata={}, page_content='Проходной балл')]